# Connect to Exasol from AzureML

In this Tutorial we will:
 - create blob storage
 - connect to Exasol SaaS
 - load tables in AzureML
 - save tables to blob storage as csv files


## Prerequisites

You will need:
 - your running Exasol Saas Cluster wih your data loaded into it
 - an AzureML account and Azure Storage account
 - authentication info for your Exasol Saas Cluster
 (..)

## Why blob storage is necessary
(explanation)
(word on data duplication?)

## AzureML setup


    - create
        - resource group,
        - workspace,
        - launch  azML studio
        - compute instance
        - notebook (this notebook) link your compute
    - in compute find public ip (if load data was run from inside azurML skip this)
    add to exasol saas like local above

"When you create a workspace, an Azure blob container and an Azure file share are automatically registered as datastores to the workspace. They're named workspaceblobstore and workspacefilestore, respectively. ""https://docs.microsoft.com/en-us/azure/machine-learning/how-to-access-data"
can find in data in azureml and in azure portal storage account in container as "azureml-blobstore-someID"

if you do not want to use default blobstore
   - create blob storage:
    - in azure portal of storage account:
        -> data storage -> containers -> + container -> in menu on left enter name -> create

        while here:     -> navigate security & networking -> access keys : remember access key and storage account name



create datastore in azureML :(this step might be moved to following tutorial or removed)
    - go to assets -> data  on th left -> datastores -> create
    menu opens on right. ender:
        - datastore name
        - datastore type (we use blob storage because the offered SQL databases are not compatible with exasol atm.)
        - subscription if needed
        - your storage account from dropdown ( from step before) pic
        - newly created blob storage/blob container from step above
        - authentication info (key from step before)
            pic




We are going to use the [PyExasol](https://docs.exasol.com/db/latest/connect_exasol/drivers/python/pyexasol.htm) package in order to connect to the Exasol database and read the data. Fist we need to install PyExasol.

In [ ]:
!pip install pyexasol

Then we need to tell pyexasol where to find our Exasol Saas Cluster with the data. Change these values to reflect your Cluster.
We ask for 10 lines of our "IDA.TEST" table to check if our connection is working.

In [ ]:
import pyexasol
import pandas
EXASOL_HOST = "<your>.clusters.exasol.com" # change
EXASOL_PORT = "8563" # change if needed
EXASOL_USER = "<your-exasol-user>" # change if needed
EXASOL_PASSWORD = "exa_pat_<your_password>" #change
EXASOL_SCHEMA = "IDA" # change if needed
EXASOL_CONNECTION = "{host}:{port}".format(host=EXASOL_HOST, port=EXASOL_PORT)
exasol = pyexasol.connect(dsn=EXASOL_CONNECTION, user=EXASOL_USER, password=EXASOL_PASSWORD, compression=True)

# check if the connection is working
data = exasol.export_to_pandas("SELECT * FROM TABLE IDA.TEST LIMIT 10")
print(data)

Next, we need to access our Azure Blob Storage Container. 

In [ ]:
from azure.ai.ml.entities import AccountKeyConfiguration

my_storage_account_name = "your_storage_account_name"
credentials= AccountKeyConfiguration(
        account_key="your_storage_account_key"
    )

explanation
mention small table size in example

In [ ]:
for table in ["TEST", "TRAIN"]:
    sql_export = f"EXPORT TABLE IDA.{table} INTO CSV AT CLOUD AZURE BLOBSTORAGE 'DefaultEndpointsProtocol=https;EndpointSuffix=core.windows.net'"\
                f"USER '{my_storage_account_name}' IDENTIFIED BY '{credentials.account_key}' FILE 'azureml-tutorial/ida/{table}'"
    exasol.execute(sql_export)


- Success!
- pic of show tables in AzureML storage